## Imports

In [2]:
import numpy as np
import pandas as pd

from astroquery.vizier import Vizier
from astropy.units import Quantity
from astroquery.gaia import Gaia

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.coordinates import Angle

import requests

## Constants and names

In [3]:
ztf_event_file_name = "ZTF_alerts.csv"

## Load file with events

In [11]:
events = pd.read_csv("ZTF_alerts.csv")

In [13]:
events.columns

Index(['#name', ' n_detections', ' first_mjd', ' last_mjd', ' ra', ' dec',
       ' class', ' hi_prob', ' delta_mjd'],
      dtype='object')

## Check Gaia catalogues

In [15]:
# Import Gaia tables and catalogues
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print (table.get_qualified_name())

INFO: Retrieving tables... [astroquery.utils.tap.core]
INFO: Parsing tables... [astroquery.utils.tap.core]
INFO: Done. [astroquery.utils.tap.core]
external.external.apassdr9
external.external.catwise2020
external.external.gaiadr2_astrophysical_parameters
external.external.gaiadr2_geometric_distance
external.external.gaiaedr3_distance
external.external.gaiaedr3_gcns_main_1
external.external.gaiaedr3_gcns_rejected_1
external.external.gaiaedr3_spurious
external.external.gaia_eso_survey
external.external.galex_ais
external.external.ravedr5_com
external.external.ravedr5_dr5
external.external.ravedr5_gra
external.external.ravedr5_on
external.external.ravedr6
external.external.sdssdr13_photoprimary
external.external.skymapperdr1_master
external.external.skymapperdr2_master
external.external.tmass_xsc
external.external.xgboost_table1
external.external.xgboost_table2
gaiadr1.gaiadr1.aux_qso_icrf2_match
gaiadr1.gaiadr1.ext_phot_zero_point
gaiadr1.gaiadr1.allwise_best_neighbour
gaiadr1.gaiadr1.al

In [43]:
# source_id, phot_g_mean_mag, bp_rp, distance_gspphot, ag_gsphot, ruwe

# For each event in ZTF list find its source id in Gaia
# Then check it in several Gaia catalogues

for index, row in events.iterrows():
    print("----------------------------------")
    print("ZTF event: ", row['#name'], "ALeRCE class: ", row[' class'])
    ra, dec = row[' ra'], row[' dec']
    adql_string = "SELECT * \
                   FROM gaiadr3.gaia_source \
                   WHERE CONTAINS(POINT(gaiadr3.gaia_source.ra, gaiadr3.gaia_source.dec),CIRCLE(%f,%f,%f))=1;"%(ra, dec, 1. / 3600.)
    
    job_single = Gaia.launch_job_async(adql_string, dump_to_file=False)
    res_single = job_single.get_results()
    if len(res_single["SOURCE_ID"]) > 0:
        source_id = res_single["SOURCE_ID"][0]

        print("RUWE: ", res_single["ruwe"][0])
        # check if appears in Gaia catalogues
        # galaxy candidates catalogue
        print("Gaia galaxy candidate:")
        adql_string = "SELECT * \
                   FROM gaiadr3.galaxy_candidates \
                   WHERE gaiadr3.galaxy_candidates.source_id=%s"%(source_id)
        job = Gaia.launch_job_async(adql_string, dump_to_file=False)
        res = job.get_results()
        if len(res['SOURCE_ID']) > 0:
            print(res["vari_best_class_name"], res["vari_best_class_score"])

        # qso candidates catalogue
        print("Gaia QSO candidate:")
        adql_string = "SELECT * \
                   FROM gaiadr3.qso_candidates \
                   WHERE gaiadr3.qso_candidates.source_id=%s"%(source_id)
        job = Gaia.launch_job_async(adql_string, dump_to_file=False)
        res = job.get_results()
        if len(res['SOURCE_ID']) > 0:
            print(res["vari_best_class_name"], res["vari_best_class_score"])

        # variability classification candidates catalogue <- used for majority of catalogues of variability
        print("Gaia variability candidate:")
        adql_string = "SELECT * \
                   FROM gaiadr3.vari_classifier_result \
                   WHERE gaiadr3.vari_classifier_result.source_id=%s"%(source_id)
        job = Gaia.launch_job_async(adql_string, dump_to_file=False)
        res = job.get_results()
        if len(res['SOURCE_ID']) > 0:
            print(res["best_class_nam"], res["best_class_score"])

----------------------------------
ZTF event:  ZTF23aagpize ALeRCE class:  	AGN
INFO: Query finished. [astroquery.utils.tap.core]
----------------------------------
ZTF event:  ZTF23aajfydl ALeRCE class:  	VS
INFO: Query finished. [astroquery.utils.tap.core]
----------------------------------
ZTF event:  ZTF23aagvqoi ALeRCE class:  	VS
INFO: Query finished. [astroquery.utils.tap.core]
----------------------------------
ZTF event:  ZTF23aaiqvoy ALeRCE class:  	bogus
INFO: Query finished. [astroquery.utils.tap.core]
----------------------------------
ZTF event:  ZTF20actltga ALeRCE class:  	bogus
INFO: Query finished. [astroquery.utils.tap.core]
RUWE:  1.1208862
Gaia galaxy candidate:
INFO: Query finished. [astroquery.utils.tap.core]
Gaia QSO candidate:
INFO: Query finished. [astroquery.utils.tap.core]
Gaia variability candidate:
INFO: Query finished. [astroquery.utils.tap.core]
----------------------------------
ZTF event:  ZTF21abwqhnf ALeRCE class:  	bogus
INFO: Query finished. [astro

In [42]:
for index, row in events.iterrows():
    print("----------------------------------")
    print("ZTF event: ", row['#name'], "ALeRCE class: ", row[' class'])
    ra, dec = row[' ra'], row[' dec']
    
    # check catalogues available through Vizier
    coord = SkyCoord(ra=ra, dec=dec, unit=(u.degree, u.degree), frame='icrs')
    # check YSO catalogues
    print("YSO candidate catalogues:")
    # check if in Konkoly YSO catalogue, Marton et al. 2023
    result1 = Vizier.query_region(coord, radius=Angle(1. / 60. / 60., "deg"), catalog='J/A+A/674/A21/kyso')
    # Marton et al 2019 YSOs
    result2 = Vizier.query_region(coord, radius=Angle(1. / 60. / 60., "deg"), catalog='II/360/catalog')

    if len(result1) > 0:
        print("Matched to a source in Konkoly YSO catalogue.")
    elif len(result2) > 0:
        table = result2[0]
        for k in range(len(table)):
            ly = table['LY'].data.data[k]
            sy = table['SY'].data.data[k]
            if (ly>0.9 or sy>0.9):
                print("High probability candidate in Gaia DR2 x WISE catalogue")

    # Query SPICY catalogue, Kuhn et al. 2020, https://ui.adsabs.harvard.edu/abs/2021ApJS..254...33K/abstract
    result1 = Vizier.query_region(coord, radius=Angle(1. / 60. / 60., "deg"), catalog='J/ApJS/254/33')
    if(len(result1) > 0):
        print(result1)

    # check GLADE+ catalogue
    print("GLADE+ catalogue:")
    # check if near galaxy in GLADE+ catalogue
    result = Vizier.query_region(coord, radius=Angle(1.5 / 60. / 60., "deg"), catalog='VII/281')
    if len(result) > 0:
        print("Within 1.5 arcsec of a known galaxy.")

    # check Miliquas catalogue
    # check if in Flesch et al. 2021 (Milliquas)
    print("Miliquas catalogue:")
    result1 = Vizier.query_region(coord, radius=Angle(2. / 60. / 60., "deg"), catalog='VII/290/catalog')
    if len(result1) > 0:
        print("Within 2 arcsec of a known QSO.")

    # check OGLE-IV Miras catalogue
    print("OGLE-IV Miras catalogue:")
    result1 = Vizier.query_region(coord, radius=Angle(1. / 60. / 60., "deg"), catalog='J/ApJS/260/46')
    if len(result1) > 0:
        print("Within 1 arcsec of a known OGLE Mira.")
    

----------------------------------
ZTF event:  ZTF23aagpize ALeRCE class:  	AGN
YSO candidate catalogues:
GLADE+ catalogue:
Miliquas catalogue:
OGLE-IV Miras catalogue:
----------------------------------
ZTF event:  ZTF23aajfydl ALeRCE class:  	VS
YSO candidate catalogues:
GLADE+ catalogue:
Miliquas catalogue:
OGLE-IV Miras catalogue:
----------------------------------
ZTF event:  ZTF23aagvqoi ALeRCE class:  	VS
YSO candidate catalogues:
GLADE+ catalogue:
Miliquas catalogue:
OGLE-IV Miras catalogue:
----------------------------------
ZTF event:  ZTF23aaiqvoy ALeRCE class:  	bogus
YSO candidate catalogues:
GLADE+ catalogue:
Miliquas catalogue:
OGLE-IV Miras catalogue:
----------------------------------
ZTF event:  ZTF20actltga ALeRCE class:  	bogus
YSO candidate catalogues:
GLADE+ catalogue:
Miliquas catalogue:
OGLE-IV Miras catalogue:
----------------------------------
ZTF event:  ZTF21abwqhnf ALeRCE class:  	bogus
YSO candidate catalogues:
GLADE+ catalogue:
Miliquas catalogue:
OGLE-IV